# DataFrame Merges: Inner, Outer, Left, and Right Joins
## 1. Setup and Sample Data

In [2]:
import pandas as pd
import numpy as np

print("CREATING SAMPLE DATASETS FOR JOIN OPERATIONS")
print("="*80)

# Dataset 1: Employee Information
employees = pd.DataFrame({
    'employee_id': [101, 102, 103, 104, 105, 106],
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank'],
    'department_id': [1, 2, 1, 3, 2, 4],
    'hire_date': ['2020-01-15', '2019-03-20', '2021-07-10', 
                  '2022-02-01', '2023-05-15', '2021-11-30'],
    'salary': [75000, 85000, 65000, 78000, 60000, 90000]
})

# Dataset 2: Department Information
departments = pd.DataFrame({
    'department_id': [1, 2, 3, 5],  # Note: id 4 missing, id 5 extra
    'department_name': ['IT', 'HR', 'Finance', 'Marketing'],
    'location': ['NYC', 'LA', 'Chicago', 'Miami'],
    'manager_id': [101, 102, 104, 107]
})

# Dataset 3: Performance Reviews
performance = pd.DataFrame({
    'employee_id': [101, 102, 104, 105, 107, 108],  # Note: 103, 106 missing; 107, 108 extra
    'review_date': ['2024-01-15', '2024-01-20', '2024-02-01', 
                    '2024-02-10', '2024-01-25', '2024-02-05'],
    'rating': [4.5, 4.2, 3.8, 4.0, 4.7, 3.5],
    'bonus_amount': [5000, 4500, 3000, 3500, 6000, 2500]
})

# Dataset 4: Projects Assignment
projects = pd.DataFrame({
    'project_id': ['P001', 'P002', 'P003', 'P004'],
    'project_name': ['Website Redesign', 'Payroll System', 'Market Analysis', 'Mobile App'],
    'lead_employee_id': [101, 102, 104, 110],  # Note: 110 not in employees
    'budget': [50000, 75000, 30000, 100000]
})

print("EMPLOYEES DataFrame:")
print(employees)
print(f"\nShape: {employees.shape}, IDs: {employees['employee_id'].tolist()}")

print("\n\nDEPARTMENTS DataFrame:")
print(departments)
print(f"\nShape: {departments.shape}, IDs: {departments['department_id'].tolist()}")

print("\n\nPERFORMANCE DataFrame:")
print(performance)
print(f"\nShape: {performance.shape}, IDs: {performance['employee_id'].tolist()}")

print("\n\nPROJECTS DataFrame:")
print(projects)
print(f"\nShape: {projects.shape}, Lead IDs: {projects['lead_employee_id'].tolist()}")

CREATING SAMPLE DATASETS FOR JOIN OPERATIONS
EMPLOYEES DataFrame:
   employee_id     name  department_id   hire_date  salary
0          101    Alice              1  2020-01-15   75000
1          102      Bob              2  2019-03-20   85000
2          103  Charlie              1  2021-07-10   65000
3          104    David              3  2022-02-01   78000
4          105      Eva              2  2023-05-15   60000
5          106    Frank              4  2021-11-30   90000

Shape: (6, 5), IDs: [101, 102, 103, 104, 105, 106]


DEPARTMENTS DataFrame:
   department_id department_name location  manager_id
0              1              IT      NYC         101
1              2              HR       LA         102
2              3         Finance  Chicago         104
3              5       Marketing    Miami         107

Shape: (4, 4), IDs: [1, 2, 3, 5]


PERFORMANCE DataFrame:
   employee_id review_date  rating  bonus_amount
0          101  2024-01-15     4.5          5000
1          102  2

# 2. INNER JOIN
## Example 2.1: Basic Inner Join

In [3]:
print("\n" + "="*80)
print("INNER JOIN EXAMPLES")
print("="*80)

# Basic inner join - employees with departments
inner_join = pd.merge(employees, departments, 
                      on='department_id', 
                      how='inner')
print("1. Employees INNER JOIN Departments:")
print(inner_join[['employee_id', 'name', 'department_id', 'department_name', 'location']])
print(f"\nResult shape: {inner_join.shape}")
print("Note: Employee 106 (dept_id=4) and Department 5 are excluded")

# Inner join with multiple keys
inner_multi = pd.merge(employees, performance,
                       on='employee_id',
                       how='inner')
print("\n2. Employees INNER JOIN Performance:")
print(inner_multi[['employee_id', 'name', 'review_date', 'rating', 'bonus_amount']])
print(f"\nResult shape: {inner_multi.shape}")
print("Note: Only employees with performance reviews (101, 102, 104, 105)")

# Inner join with different column names
print("\n3. Projects INNER JOIN Employees (different column names):")
projects_renamed = projects.rename(columns={'lead_employee_id': 'employee_id'})
inner_projects = pd.merge(employees, projects_renamed,
                          on='employee_id',
                          how='inner')
print(inner_projects[['employee_id', 'name', 'project_name', 'budget']])
print(f"\nResult shape: {inner_projects.shape}")
print("Note: Only employees who lead projects (101, 102, 104)")


INNER JOIN EXAMPLES
1. Employees INNER JOIN Departments:
   employee_id     name  department_id department_name location
0          101    Alice              1              IT      NYC
1          102      Bob              2              HR       LA
2          103  Charlie              1              IT      NYC
3          104    David              3         Finance  Chicago
4          105      Eva              2              HR       LA

Result shape: (5, 8)
Note: Employee 106 (dept_id=4) and Department 5 are excluded

2. Employees INNER JOIN Performance:
   employee_id   name review_date  rating  bonus_amount
0          101  Alice  2024-01-15     4.5          5000
1          102    Bob  2024-01-20     4.2          4500
2          104  David  2024-02-01     3.8          3000
3          105    Eva  2024-02-10     4.0          3500

Result shape: (4, 8)
Note: Only employees with performance reviews (101, 102, 104, 105)

3. Projects INNER JOIN Employees (different column names):
   emplo

# Example 2.2: Inner Join with Suffixes

In [4]:
# Create data with overlapping column names
employee_contact = pd.DataFrame({
    'employee_id': [101, 102, 103, 104, 105],
    'contact_info': ['alice@company.com', 'bob@company.com', 
                     'charlie@company.com', 'david@company.com', 
                     'eva@company.com'],
    'phone': ['555-0101', '555-0102', '555-0103', '555-0104', '555-0105']
})

employee_emergency = pd.DataFrame({
    'employee_id': [101, 102, 103, 104],
    'contact_info': ['John Smith', 'Jane Johnson', 'Mary Brown', 'Tom Wilson'],
    'phone': ['555-0201', '555-0202', '555-0203', '555-0204']
})

print("\n4. Inner Join with overlapping column names (using suffixes):")
inner_suffix = pd.merge(employee_contact, employee_emergency,
                        on='employee_id',
                        how='inner',
                        suffixes=('_work', '_emergency'))
print(inner_suffix)
print(f"\nNote: Suffixes added to distinguish overlapping columns")


4. Inner Join with overlapping column names (using suffixes):
   employee_id    contact_info_work phone_work contact_info_emergency  \
0          101    alice@company.com   555-0101             John Smith   
1          102      bob@company.com   555-0102           Jane Johnson   
2          103  charlie@company.com   555-0103             Mary Brown   
3          104    david@company.com   555-0104             Tom Wilson   

  phone_emergency  
0        555-0201  
1        555-0202  
2        555-0203  
3        555-0204  

Note: Suffixes added to distinguish overlapping columns


#  3. LEFT JOIN
## Example 3.1: Basic Left Join

In [5]:
print("\n" + "="*80)
print("LEFT JOIN EXAMPLES")
print("="*80)

# Left join - all employees with their department info
left_join = pd.merge(employees, departments,
                     on='department_id',
                     how='left')
print("1. Employees LEFT JOIN Departments (all employees, department info if available):")
print(left_join[['employee_id', 'name', 'department_id', 'department_name', 'location']])
print(f"\nResult shape: {left_join.shape}")
print("Note: Employee 106 has NULL for department info (dept_id=4 doesn't exist)")

# Left join with performance data
print("\n2. Employees LEFT JOIN Performance (all employees, performance if available):")
left_performance = pd.merge(employees, performance,
                            on='employee_id',
                            how='left')
print(left_performance[['employee_id', 'name', 'review_date', 'rating', 'bonus_amount']])
print(f"\nResult shape: {left_performance.shape}")
print("Note: Employees 103 and 106 have NULL for performance data")


LEFT JOIN EXAMPLES
1. Employees LEFT JOIN Departments (all employees, department info if available):
   employee_id     name  department_id department_name location
0          101    Alice              1              IT      NYC
1          102      Bob              2              HR       LA
2          103  Charlie              1              IT      NYC
3          104    David              3         Finance  Chicago
4          105      Eva              2              HR       LA
5          106    Frank              4             NaN      NaN

Result shape: (6, 8)
Note: Employee 106 has NULL for department info (dept_id=4 doesn't exist)

2. Employees LEFT JOIN Performance (all employees, performance if available):
   employee_id     name review_date  rating  bonus_amount
0          101    Alice  2024-01-15     4.5        5000.0
1          102      Bob  2024-01-20     4.2        4500.0
2          103  Charlie         NaN     NaN           NaN
3          104    David  2024-02-01     3.8

# Example 3.2: Left Join with Filtering

In [6]:
# Find employees without performance reviews
employees_no_review = left_performance[left_performance['review_date'].isna()]
print("\n3. Employees WITHOUT performance reviews (using LEFT JOIN):")
print(employees_no_review[['employee_id', 'name', 'department_id']])

# Left join with indicator
print("\n4. Left Join with indicator (shows source of each row):")
left_with_indicator = pd.merge(employees, performance,
                               on='employee_id',
                               how='left',
                               indicator=True)
print(left_with_indicator[['employee_id', 'name', 'review_date', 'rating', '_merge']])
print("\n_merge column values:")
print("- 'both': Employee has performance review")
print("- 'left_only': Employee has NO performance review")


3. Employees WITHOUT performance reviews (using LEFT JOIN):
   employee_id     name  department_id
2          103  Charlie              1
5          106    Frank              4

4. Left Join with indicator (shows source of each row):
   employee_id     name review_date  rating     _merge
0          101    Alice  2024-01-15     4.5       both
1          102      Bob  2024-01-20     4.2       both
2          103  Charlie         NaN     NaN  left_only
3          104    David  2024-02-01     3.8       both
4          105      Eva  2024-02-10     4.0       both
5          106    Frank         NaN     NaN  left_only

_merge column values:
- 'both': Employee has performance review
- 'left_only': Employee has NO performance review


# 4. RIGHT JOIN
## Example 4.1: Basic Right Join

In [7]:
print("\n" + "="*80)
print("RIGHT JOIN EXAMPLES")
print("="*80)

# Right join - all departments with their employees
right_join = pd.merge(employees, departments,
                      on='department_id',
                      how='right')
print("1. Employees RIGHT JOIN Departments (all departments, employees if exist):")
print(right_join[['employee_id', 'name', 'department_id', 'department_name', 'location']])
print(f"\nResult shape: {right_join.shape}")
print("Note: Department 5 (Marketing) has NULL employee info")

# Right join is essentially a left join with tables swapped
print("\n2. Demonstrating RIGHT JOIN is LEFT JOIN with tables swapped:")
right_vs_left = pd.merge(departments, employees,
                         on='department_id',
                         how='left')
print("Departments LEFT JOIN Employees (same result as Employees RIGHT JOIN Departments):")
print(right_vs_left[['department_id', 'department_name', 'employee_id', 'name']])
print(f"\nSame shape: {right_join.shape} == {right_vs_left.shape}")


RIGHT JOIN EXAMPLES
1. Employees RIGHT JOIN Departments (all departments, employees if exist):
   employee_id     name  department_id department_name location
0        101.0    Alice              1              IT      NYC
1        103.0  Charlie              1              IT      NYC
2        102.0      Bob              2              HR       LA
3        105.0      Eva              2              HR       LA
4        104.0    David              3         Finance  Chicago
5          NaN      NaN              5       Marketing    Miami

Result shape: (6, 8)
Note: Department 5 (Marketing) has NULL employee info

2. Demonstrating RIGHT JOIN is LEFT JOIN with tables swapped:
Departments LEFT JOIN Employees (same result as Employees RIGHT JOIN Departments):
   department_id department_name  employee_id     name
0              1              IT        101.0    Alice
1              1              IT        103.0  Charlie
2              2              HR        102.0      Bob
3             

# Example 4.2: Right Join Use Cases

In [8]:
# Find departments without employees
departments_no_employees = right_join[right_join['employee_id'].isna()]
print("\n3. Departments WITHOUT employees (using RIGHT JOIN):")
print(departments_no_employees[['department_id', 'department_name', 'location']])

# Right join with performance data
print("\n4. Performance RIGHT JOIN Employees (all performance records, employee info if exists):")
right_performance = pd.merge(employees, performance,
                             on='employee_id',
                             how='right')
print(right_performance[['employee_id', 'name', 'review_date', 'rating', 'bonus_amount']])
print(f"\nResult shape: {right_performance.shape}")
print("Note: Performance records for employees 107 and 108 have NULL employee info")


3. Departments WITHOUT employees (using RIGHT JOIN):
   department_id department_name location
5              5       Marketing    Miami

4. Performance RIGHT JOIN Employees (all performance records, employee info if exists):
   employee_id   name review_date  rating  bonus_amount
0          101  Alice  2024-01-15     4.5          5000
1          102    Bob  2024-01-20     4.2          4500
2          104  David  2024-02-01     3.8          3000
3          105    Eva  2024-02-10     4.0          3500
4          107    NaN  2024-01-25     4.7          6000
5          108    NaN  2024-02-05     3.5          2500

Result shape: (6, 8)
Note: Performance records for employees 107 and 108 have NULL employee info


# 5. OUTER JOIN (FULL OUTER JOIN)
## Example 5.1: Basic Outer Join

In [9]:
print("\n" + "="*80)
print("OUTER JOIN (FULL JOIN) EXAMPLES")
print("="*80)

# Outer join - all employees and all departments
outer_join = pd.merge(employees, departments,
                      on='department_id',
                      how='outer')
print("1. Employees OUTER JOIN Departments (all records from both tables):")
print(outer_join[['employee_id', 'name', 'department_id', 'department_name', 'location']])
print(f"\nResult shape: {outer_join.shape}")
print("Note: Includes Employee 106 (no department) AND Department 5 (no employees)")

# Outer join with indicator
print("\n2. Outer Join with indicator:")
outer_with_indicator = pd.merge(employees, departments,
                                on='department_id',
                                how='outer',
                                indicator=True)
print(outer_with_indicator[['employee_id', 'name', 'department_id', 'department_name', '_merge']])
print("\n_merge column analysis:")
merge_counts = outer_with_indicator['_merge'].value_counts()
for value, count in merge_counts.items():
    print(f"  {value}: {count} records")


OUTER JOIN (FULL JOIN) EXAMPLES
1. Employees OUTER JOIN Departments (all records from both tables):
   employee_id     name  department_id department_name location
0        101.0    Alice              1              IT      NYC
1        103.0  Charlie              1              IT      NYC
2        102.0      Bob              2              HR       LA
3        105.0      Eva              2              HR       LA
4        104.0    David              3         Finance  Chicago
5        106.0    Frank              4             NaN      NaN
6          NaN      NaN              5       Marketing    Miami

Result shape: (7, 8)
Note: Includes Employee 106 (no department) AND Department 5 (no employees)

2. Outer Join with indicator:
   employee_id     name  department_id department_name      _merge
0        101.0    Alice              1              IT        both
1        103.0  Charlie              1              IT        both
2        102.0      Bob              2              HR   

# Example 5.2: Outer Join for Data Reconciliation

In [10]:
# Data reconciliation example
print("\n3. Data Reconciliation using Outer Join:")
print("Identifying data gaps between Employees and Performance tables")

reconciliation = pd.merge(employees[['employee_id', 'name']], 
                          performance[['employee_id', 'rating']],
                          on='employee_id',
                          how='outer',
                          indicator=True)

print("\nReconciliation Results:")
print(reconciliation.sort_values('employee_id'))

# Categorize the results
print("\n\nData Gap Analysis:")
only_in_employees = reconciliation[reconciliation['_merge'] == 'left_only']
only_in_performance = reconciliation[reconciliation['_merge'] == 'right_only']
in_both = reconciliation[reconciliation['_merge'] == 'both']

print(f"Employees without performance reviews: {len(only_in_employees)}")
print(f"Performance reviews without employee records: {len(only_in_performance)}")
print(f"Employees with performance reviews: {len(in_both)}")


3. Data Reconciliation using Outer Join:
Identifying data gaps between Employees and Performance tables

Reconciliation Results:
   employee_id     name  rating      _merge
0          101    Alice     4.5        both
1          102      Bob     4.2        both
2          103  Charlie     NaN   left_only
3          104    David     3.8        both
4          105      Eva     4.0        both
5          106    Frank     NaN   left_only
6          107      NaN     4.7  right_only
7          108      NaN     3.5  right_only


Data Gap Analysis:
Employees without performance reviews: 2
Performance reviews without employee records: 2
Employees with performance reviews: 4


# 6. Real-World Business Examples
# Example 6.1: E-commerce Order Analysis

In [11]:
print("\n" + "="*80)
print("REAL-WORLD EXAMPLE: E-COMMERCE ORDER ANALYSIS")
print("="*80)

# Create e-commerce datasets
customers = pd.DataFrame({
    'customer_id': ['C001', 'C002', 'C003', 'C004', 'C005'],
    'customer_name': ['Alice', 'Bob', 'Charlie', 'Diana', 'Evan'],
    'join_date': ['2023-01-15', '2023-02-20', '2023-03-10', '2023-04-05', '2023-05-12'],
    'customer_tier': ['Gold', 'Silver', 'Bronze', 'Silver', 'Bronze']
})

orders = pd.DataFrame({
    'order_id': ['O1001', 'O1002', 'O1003', 'O1004', 'O1005', 'O1006', 'O1007'],
    'customer_id': ['C001', 'C001', 'C002', 'C003', 'C004', 'C006', 'C007'],  # C006, C007 not in customers
    'order_date': ['2024-01-10', '2024-02-15', '2024-01-22', '2024-02-01', 
                   '2024-02-20', '2024-02-25', '2024-03-01'],
    'order_amount': [150.50, 89.99, 299.99, 45.50, 599.99, 129.99, 79.99],
    'status': ['Delivered', 'Shipped', 'Delivered', 'Processing', 
               'Delivered', 'Shipped', 'Processing']
})

products = pd.DataFrame({
    'product_id': ['P001', 'P002', 'P003', 'P004', 'P005'],
    'product_name': ['Laptop', 'Mouse', 'Keyboard', 'Monitor', 'Headphones'],
    'category': ['Electronics', 'Accessories', 'Accessories', 'Electronics', 'Audio'],
    'price': [999.99, 29.99, 79.99, 299.99, 149.99]
})

order_items = pd.DataFrame({
    'order_id': ['O1001', 'O1001', 'O1002', 'O1003', 'O1003', 'O1004', 'O1005', 'O1006'],
    'product_id': ['P001', 'P002', 'P003', 'P001', 'P004', 'P005', 'P001', 'P003'],
    'quantity': [1, 2, 1, 1, 2, 1, 1, 3],
    'unit_price': [999.99, 29.99, 79.99, 999.99, 299.99, 149.99, 999.99, 79.99]
})

print("E-commerce Data Overview:")
print(f"Customers: {len(customers)}")
print(f"Orders: {len(orders)}")
print(f"Products: {len(products)}")
print(f"Order Items: {len(order_items)}")

# Business Question 1: Customer Order History
print("\n1. CUSTOMER ORDER HISTORY (Customers LEFT JOIN Orders):")
customer_orders = pd.merge(customers, orders,
                           on='customer_id',
                           how='left')
print(customer_orders[['customer_id', 'customer_name', 'order_id', 'order_date', 'order_amount']])
print(f"\nCustomers without orders: {customer_orders['order_id'].isna().sum()}")

# Business Question 2: Complete Order Details
print("\n2. COMPLETE ORDER DETAILS (Multiple Joins):")
# Step 1: Orders with customer info
orders_with_customers = pd.merge(orders, customers,
                                 on='customer_id',
                                 how='left')

# Step 2: Add order items
orders_with_items = pd.merge(orders_with_customers, order_items,
                             on='order_id',
                             how='left')

# Step 3: Add product info
complete_orders = pd.merge(orders_with_items, products,
                           on='product_id',
                           how='left')

# Calculate line total
complete_orders['line_total'] = complete_orders['quantity'] * complete_orders['unit_price']

print("Sample of complete order details:")
print(complete_orders[['order_id', 'customer_name', 'product_name', 
                       'quantity', 'unit_price', 'line_total']].head(10))

# Business Question 3: Products never ordered
print("\n3. PRODUCTS NEVER ORDERED (Products LEFT JOIN Order Items):")
products_ordered = pd.merge(products, order_items,
                            on='product_id',
                            how='left',
                            indicator=True)

never_ordered = products_ordered[products_ordered['_merge'] == 'left_only']
print("Products that have never been ordered:")
print(never_ordered[['product_id', 'product_name', 'category', 'price']])


REAL-WORLD EXAMPLE: E-COMMERCE ORDER ANALYSIS
E-commerce Data Overview:
Customers: 5
Orders: 7
Products: 5
Order Items: 8

1. CUSTOMER ORDER HISTORY (Customers LEFT JOIN Orders):
  customer_id customer_name order_id  order_date  order_amount
0        C001         Alice    O1001  2024-01-10        150.50
1        C001         Alice    O1002  2024-02-15         89.99
2        C002           Bob    O1003  2024-01-22        299.99
3        C003       Charlie    O1004  2024-02-01         45.50
4        C004         Diana    O1005  2024-02-20        599.99
5        C005          Evan      NaN         NaN           NaN

Customers without orders: 1

2. COMPLETE ORDER DETAILS (Multiple Joins):
Sample of complete order details:
  order_id customer_name product_name  quantity  unit_price  line_total
0    O1001         Alice       Laptop       1.0      999.99      999.99
1    O1001         Alice        Mouse       2.0       29.99       59.98
2    O1002         Alice     Keyboard       1.0       7

# Example 6.2: HR Analytics Dashboard

In [12]:
print("\n" + "="*80)
print("REAL-WORLD EXAMPLE: HR ANALYTICS DASHBOARD")
print("="*80)

# HR datasets
employees_hr = pd.DataFrame({
    'emp_id': [101, 102, 103, 104, 105, 106, 107],
    'emp_name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank', 'Grace'],
    'dept_id': ['D001', 'D002', 'D001', 'D003', 'D002', 'D004', 'D005'],
    'position': ['Manager', 'Analyst', 'Developer', 'Manager', 'Analyst', 'Developer', 'HR Specialist'],
    'salary': [85000, 65000, 75000, 90000, 62000, 72000, 58000]
})

departments_hr = pd.DataFrame({
    'dept_id': ['D001', 'D002', 'D003', 'D004'],
    'dept_name': ['IT', 'Finance', 'Marketing', 'Operations'],
    'location': ['HQ', 'HQ', 'Branch A', 'HQ'],
    'budget': [500000, 300000, 200000, 400000]
})

attendance = pd.DataFrame({
    'emp_id': [101, 101, 102, 103, 104, 105, 106, 107, 108],
    'date': ['2024-01-15', '2024-01-16', '2024-01-15', '2024-01-15',
             '2024-01-15', '2024-01-15', '2024-01-15', '2024-01-15', '2024-01-15'],
    'status': ['Present', 'Present', 'Present', 'Late', 'Present', 
               'Absent', 'Present', 'Present', 'Present'],
    'hours_worked': [8, 8, 7.5, 6, 8, 0, 8, 8, 8]
})

training = pd.DataFrame({
    'training_id': ['T001', 'T002', 'T003', 'T004'],
    'training_name': ['Leadership', 'Excel Advanced', 'Python Basics', 'Communication'],
    'emp_id': [101, 102, 103, 105],
    'completion_date': ['2023-12-15', '2024-01-10', '2024-01-20', '2024-02-01'],
    'score': [95, 88, 92, 85]
})

print("HR Data Overview:")
print(f"Employees: {len(employees_hr)}")
print(f"Departments: {len(departments_hr)}")
print(f"Attendance records: {len(attendance)}")
print(f"Training records: {len(training)}")

# Analysis 1: Department Headcount and Budget Analysis
print("\n1. DEPARTMENT HEADCOUNT AND BUDGET (LEFT JOIN):")
dept_analysis = pd.merge(departments_hr, employees_hr,
                         on='dept_id',
                         how='left')

# Calculate department stats
dept_summary = dept_analysis.groupby(['dept_id', 'dept_name', 'budget']).agg(
    headcount=('emp_id', 'count'),
    avg_salary=('salary', 'mean'),
    total_salary=('salary', 'sum')
).reset_index()

dept_summary['salary_to_budget_ratio'] = dept_summary['total_salary'] / dept_summary['budget']

print("Department Summary:")
print(dept_summary.to_string(index=False))

# Analysis 2: Employee Master View
print("\n2. EMPLOYEE MASTER VIEW (Multiple LEFT JOINS):")
employee_master = pd.merge(employees_hr, departments_hr,
                           on='dept_id',
                           how='left')

# Add attendance summary
attendance_summary = attendance.groupby('emp_id').agg(
    total_days=('date', 'count'),
    avg_hours=('hours_worked', 'mean'),
    attendance_rate=('status', lambda x: (x == 'Present').mean() * 100)
).reset_index()

employee_master = pd.merge(employee_master, attendance_summary,
                           left_on='emp_id',
                           right_on='emp_id',
                           how='left')

# Add training summary
training_summary = training.groupby('emp_id').agg(
    trainings_completed=('training_id', 'count'),
    avg_training_score=('score', 'mean')
).reset_index()

employee_master = pd.merge(employee_master, training_summary,
                           on='emp_id',
                           how='left')

print("Employee Master View (first 5 employees):")
print(employee_master.head().to_string(index=False))

# Analysis 3: Find gaps - Employees without department or attendance
print("\n3. DATA GAP ANALYSIS (OUTER JOINS with indicators):")

# Employees without department
emp_dept_gap = pd.merge(employees_hr, departments_hr,
                        on='dept_id',
                        how='outer',
                        indicator=True)

print("Employees without department assignments:")
no_dept = emp_dept_gap[emp_dept_gap['_merge'] == 'left_only']
if len(no_dept) > 0:
    print(no_dept[['emp_id', 'emp_name', 'dept_id']])
else:
    print("All employees have department assignments")

print("\nDepartments without employees:")
no_employees = emp_dept_gap[emp_dept_gap['_merge'] == 'right_only']
if len(no_employees) > 0:
    print(no_employees[['dept_id', 'dept_name']])
else:
    print("All departments have employees")


REAL-WORLD EXAMPLE: HR ANALYTICS DASHBOARD
HR Data Overview:
Employees: 7
Departments: 4
Attendance records: 9
Training records: 4

1. DEPARTMENT HEADCOUNT AND BUDGET (LEFT JOIN):
Department Summary:
dept_id  dept_name  budget  headcount  avg_salary  total_salary  salary_to_budget_ratio
   D001         IT  500000          2     80000.0        160000                0.320000
   D002    Finance  300000          2     63500.0        127000                0.423333
   D003  Marketing  200000          1     90000.0         90000                0.450000
   D004 Operations  400000          1     72000.0         72000                0.180000

2. EMPLOYEE MASTER VIEW (Multiple LEFT JOINS):
Employee Master View (first 5 employees):
 emp_id emp_name dept_id  position  salary dept_name location   budget  total_days  avg_hours  attendance_rate  trainings_completed  avg_training_score
    101    Alice    D001   Manager   85000        IT       HQ 500000.0           2        8.0            100.0       

# 7. Advanced Join Techniques
## Example 7.1: Joining on Multiple Columns

In [13]:
print("\n" + "="*80)
print("ADVANCED: JOINING ON MULTIPLE COLUMNS")
print("="*80)

# Sales data with composite key
sales_q1 = pd.DataFrame({
    'region': ['North', 'North', 'South', 'South', 'East'],
    'product': ['A', 'B', 'A', 'C', 'B'],
    'quarter': ['Q1', 'Q1', 'Q1', 'Q1', 'Q1'],
    'sales': [1000, 1500, 800, 1200, 900],
    'target': [900, 1400, 850, 1100, 950]
})

sales_q2 = pd.DataFrame({
    'region': ['North', 'North', 'South', 'West', 'East'],
    'product': ['A', 'B', 'A', 'C', 'B'],
    'quarter': ['Q2', 'Q2', 'Q2', 'Q2', 'Q2'],
    'sales': [1100, 1600, 850, 1300, 950],
    'target': [1000, 1500, 800, 1200, 900]
})

print("Q1 Sales:")
print(sales_q1)
print("\nQ2 Sales:")
print(sales_q2)

# Join on multiple columns
print("\n1. Sales Comparison Q1 vs Q2 (Join on region AND product):")
sales_comparison = pd.merge(sales_q1, sales_q2,
                            on=['region', 'product'],
                            how='outer',
                            suffixes=('_q1', '_q2'))
print(sales_comparison)

# Calculate growth
sales_comparison['sales_growth'] = (
    (sales_comparison['sales_q2'] - sales_comparison['sales_q1']) / 
    sales_comparison['sales_q1'] * 100
)
sales_comparison['target_achievement_q1'] = (
    sales_comparison['sales_q1'] / sales_comparison['target_q1'] * 100
)
sales_comparison['target_achievement_q2'] = (
    sales_comparison['sales_q2'] / sales_comparison['target_q2'] * 100
)

print("\n2. Sales Growth and Target Achievement:")
print(sales_comparison[['region', 'product', 'sales_q1', 'sales_q2', 
                        'sales_growth', 'target_achievement_q1', 
                        'target_achievement_q2']].round(2))


ADVANCED: JOINING ON MULTIPLE COLUMNS
Q1 Sales:
  region product quarter  sales  target
0  North       A      Q1   1000     900
1  North       B      Q1   1500    1400
2  South       A      Q1    800     850
3  South       C      Q1   1200    1100
4   East       B      Q1    900     950

Q2 Sales:
  region product quarter  sales  target
0  North       A      Q2   1100    1000
1  North       B      Q2   1600    1500
2  South       A      Q2    850     800
3   West       C      Q2   1300    1200
4   East       B      Q2    950     900

1. Sales Comparison Q1 vs Q2 (Join on region AND product):
  region product quarter_q1  sales_q1  target_q1 quarter_q2  sales_q2  \
0   East       B         Q1     900.0      950.0         Q2     950.0   
1  North       A         Q1    1000.0      900.0         Q2    1100.0   
2  North       B         Q1    1500.0     1400.0         Q2    1600.0   
3  South       A         Q1     800.0      850.0         Q2     850.0   
4  South       C         Q1    1200

# Example 7.2: Non-Equi Joins (using cross join and filter)

In [14]:
print("\n" + "="*80)
print("ADVANCED: NON-EQUI JOINS (Salary Bands)")
print("="*80)

# Salary bands for positions
salary_bands = pd.DataFrame({
    'position': ['Junior', 'Mid', 'Senior', 'Lead', 'Manager'],
    'min_salary': [40000, 60000, 80000, 100000, 120000],
    'max_salary': [59999, 79999, 99999, 119999, 150000]
})

# Employee current salaries
employee_salaries = pd.DataFrame({
    'emp_id': [101, 102, 103, 104, 105, 106],
    'name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank'],
    'current_salary': [85000, 65000, 95000, 110000, 45000, 125000],
    'current_title': ['Senior Developer', 'Data Analyst', 'Lead Engineer', 
                      'Manager', 'Junior Analyst', 'Director']
})

print("Salary Bands:")
print(salary_bands)
print("\nEmployee Salaries:")
print(employee_salaries)

# Method 1: Cross join + filter (for non-equi join)
print("\n1. Finding salary bands for employees (Cross Join + Filter):")
# Create cartesian product
cross_join = pd.merge(employee_salaries.assign(key=1), 
                      salary_bands.assign(key=1),
                      on='key').drop('key', axis=1)

# Filter where salary falls within band
salary_band_analysis = cross_join[
    (cross_join['current_salary'] >= cross_join['min_salary']) &
    (cross_join['current_salary'] <= cross_join['max_salary'])
]

print(salary_band_analysis[['emp_id', 'name', 'current_salary', 
                            'position', 'min_salary', 'max_salary']])

# Method 2: Using pd.merge_asof (for range joins)
print("\n2. Using merge_asof for range-based joins:")
# Sort for merge_asof
salary_bands_sorted = salary_bands.sort_values('min_salary')
employee_salaries_sorted = employee_salaries.sort_values('current_salary')

# Use merge_asof for range join (requires sorted data)
band_analysis_asof = pd.merge_asof(employee_salaries_sorted,
                                   salary_bands_sorted,
                                   left_on='current_salary',
                                   right_on='min_salary',
                                   direction='forward')

# Filter where salary <= max_salary
band_analysis_asof = band_analysis_asof[
    band_analysis_asof['current_salary'] <= band_analysis_asof['max_salary']
]

print(band_analysis_asof[['emp_id', 'name', 'current_salary', 
                          'position', 'min_salary', 'max_salary']])


ADVANCED: NON-EQUI JOINS (Salary Bands)
Salary Bands:
  position  min_salary  max_salary
0   Junior       40000       59999
1      Mid       60000       79999
2   Senior       80000       99999
3     Lead      100000      119999
4  Manager      120000      150000

Employee Salaries:
   emp_id     name  current_salary     current_title
0     101    Alice           85000  Senior Developer
1     102      Bob           65000      Data Analyst
2     103  Charlie           95000     Lead Engineer
3     104    David          110000           Manager
4     105      Eva           45000    Junior Analyst
5     106    Frank          125000          Director

1. Finding salary bands for employees (Cross Join + Filter):
    emp_id     name  current_salary position  min_salary  max_salary
2      101    Alice           85000   Senior       80000       99999
6      102      Bob           65000      Mid       60000       79999
12     103  Charlie           95000   Senior       80000       99999
18    

# Example 7.3: Self Join

In [15]:
print("\n" + "="*80)
print("ADVANCED: SELF JOIN (Employee-Manager Hierarchy)")
print("="*80)

# Employee hierarchy
hierarchy = pd.DataFrame({
    'emp_id': [101, 102, 103, 104, 105, 106, 107],
    'emp_name': ['Alice', 'Bob', 'Charlie', 'David', 'Eva', 'Frank', 'Grace'],
    'manager_id': [104, 104, 106, 106, 102, 107, None],  # CEO has no manager
    'title': ['Developer', 'Analyst', 'Designer', 'Manager', 
              'Analyst', 'Director', 'CEO']
})

print("Employee Hierarchy:")
print(hierarchy)

# Self join to get manager names
print("\n1. Self Join to get Manager Names:")
employee_manager = pd.merge(hierarchy, 
                            hierarchy[['emp_id', 'emp_name']].rename(
                                columns={'emp_id': 'manager_id', 
                                        'emp_name': 'manager_name'}),
                            on='manager_id',
                            how='left')

print(employee_manager[['emp_id', 'emp_name', 'title', 
                        'manager_id', 'manager_name']])

# Find employees who are also managers
print("\n2. Employees who are Managers:")
managers = hierarchy[hierarchy['emp_id'].isin(hierarchy['manager_id'])]
print(managers[['emp_id', 'emp_name', 'title']])

# Find employees with no direct reports (leaf nodes)
print("\n3. Employees with no direct reports (Leaf Nodes):")
leaf_employees = hierarchy[~hierarchy['emp_id'].isin(hierarchy['manager_id'].dropna())]
print(leaf_employees[['emp_id', 'emp_name', 'title']])


ADVANCED: SELF JOIN (Employee-Manager Hierarchy)
Employee Hierarchy:
   emp_id emp_name  manager_id      title
0     101    Alice       104.0  Developer
1     102      Bob       104.0    Analyst
2     103  Charlie       106.0   Designer
3     104    David       106.0    Manager
4     105      Eva       102.0    Analyst
5     106    Frank       107.0   Director
6     107    Grace         NaN        CEO

1. Self Join to get Manager Names:
   emp_id emp_name      title  manager_id manager_name
0     101    Alice  Developer       104.0        David
1     102      Bob    Analyst       104.0        David
2     103  Charlie   Designer       106.0        Frank
3     104    David    Manager       106.0        Frank
4     105      Eva    Analyst       102.0          Bob
5     106    Frank   Director       107.0        Grace
6     107    Grace        CEO         NaN          NaN

2. Employees who are Managers:
   emp_id emp_name     title
1     102      Bob   Analyst
3     104    David   Manager

# 8. Performance Tips and Best Practices

In [16]:
print("\n" + "="*80)
print("PERFORMANCE TIPS AND BEST PRACTICES")
print("="*80)

import time

# Create large datasets for performance testing
np.random.seed(42)
n_rows = 100000

large_df1 = pd.DataFrame({
    'id': range(n_rows),
    'value1': np.random.randn(n_rows),
    'category': np.random.choice(['A', 'B', 'C', 'D', 'E'], n_rows)
})

large_df2 = pd.DataFrame({
    'id': range(int(n_rows * 0.8), int(n_rows * 1.8)),  # Overlapping IDs
    'value2': np.random.randn(int(n_rows * 1.0)),
    'category': np.random.choice(['A', 'B', 'C', 'D', 'E', 'F', 'G'], int(n_rows * 1.0))
})

print(f"Performance Test with {n_rows:,} rows")

# Test different join types
join_types = ['inner', 'left', 'outer']
results = {}

for join_type in join_types:
    start_time = time.time()
    result = pd.merge(large_df1, large_df2, 
                      on='id', 
                      how=join_type)
    elapsed = time.time() - start_time
    results[join_type] = {
        'time': elapsed,
        'rows': len(result),
        'cols': result.shape[1]
    }
    print(f"{join_type.upper()} JOIN: {elapsed:.3f}s, "
          f"{len(result):,} rows, {result.shape[1]} columns")

print("\n" + "="*80)
print("BEST PRACTICES:")
print("="*80)

practices = [
    ("Use appropriate join type",
     "✓ INNER: When you need matching records only\n"
     "✓ LEFT: When you need all from left table\n"
     "✓ RIGHT: When you need all from right table\n"
     "✓ OUTER: When you need all records"),
    
    ("Set indexes before joining",
     "✓ df.set_index('key_column')\n"
     "✓ Joins on index are faster than on columns"),
    
    ("Handle duplicates before joining",
     "✓ Check for duplicate keys\n"
     "✓ Use df.drop_duplicates() if needed"),
    
    ("Use suffixes for overlapping columns",
     "✓ suffixes=('_left', '_right')\n"
     "✓ Prevents column name conflicts"),
    
    ("Consider merge order for performance",
     "✓ Smaller table first can be faster\n"
     "✓ Filter data before joining when possible"),
    
    ("Use pd.merge() vs df.join()",
     "✓ pd.merge(): More features, flexible\n"
     "✓ df.join(): Simpler, for index-based joins"),
]

for title, content in practices:
    print(f"\n{title}:")
    print(content)

print("\n" + "="*80)
print("COMMON PITFALLS:")
print("="*80)

pitfalls = [
    ("Cartesian products",
     "✗ Unintended many-to-many joins\n"
     "✓ Check key uniqueness before joining"),
    
    ("Column name collisions",
     "✗ Overwriting columns\n"
     "✓ Use suffixes parameter"),
    
    ("Memory issues",
     "✗ Joining very large tables\n"
     "✓ Filter first, join in batches"),
    
    ("Wrong join type",
     "✗ Getting unexpected NULLs or missing data\n"
     "✓ Understand business logic for join type"),
    
    ("Performance with string keys",
     "✗ Joining on string columns is slower\n"
     "✓ Use categorical type for string keys"),
]

for pitfall, solution in pitfalls:
    print(f"\n{pitfall}:")
    print(solution)


PERFORMANCE TIPS AND BEST PRACTICES
Performance Test with 100,000 rows
INNER JOIN: 0.036s, 20,000 rows, 5 columns
LEFT JOIN: 0.006s, 100,000 rows, 5 columns
OUTER JOIN: 0.030s, 180,000 rows, 5 columns

BEST PRACTICES:

Use appropriate join type:
✓ INNER: When you need matching records only
✓ LEFT: When you need all from left table
✓ RIGHT: When you need all from right table
✓ OUTER: When you need all records

Set indexes before joining:
✓ df.set_index('key_column')
✓ Joins on index are faster than on columns

Handle duplicates before joining:
✓ Check for duplicate keys
✓ Use df.drop_duplicates() if needed

Use suffixes for overlapping columns:
✓ suffixes=('_left', '_right')
✓ Prevents column name conflicts

Consider merge order for performance:
✓ Smaller table first can be faster
✓ Filter data before joining when possible

Use pd.merge() vs df.join():
✓ pd.merge(): More features, flexible
✓ df.join(): Simpler, for index-based joins

COMMON PITFALLS:

Cartesian products:
✗ Unintended m